In [1]:
import pandas as pd

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn
from numba import jit

In [ ]:
tra

In [ ]:
train = pd.read_csv("/root/.kaggle/competitions/ai-hack-2018-msk-gpn/train_data.csv")
test = pd.read_csv("/root/.kaggle/competitions/ai-hack-2018-msk-gpn/test_data.csv")
sam = pd.read_csv("/root/.kaggle/competitions/ai-hack-2018-msk-gpn/sample_submission.csv")

In [3]:
train = pd.read_csv("./train_with_data.csv")
test = pd.read_csv("./test_with_data.csv")
sam = pd.read_csv("/root/.kaggle/competitions/ai-hack-2018-msk-gpn/sample_submission.csv")

In [3]:
train = pd.read_csv("./train_with_mean.csv")
test = pd.read_csv("./test_with_mean.csv")
sam = pd.read_csv("/root/.kaggle/competitions/ai-hack-2018-msk-gpn/sample_submission.csv")

In [4]:
y = train.COL_LITR
train.drop(['Unnamed: 0'], axis=1, inplace=True)
test.drop(['Unnamed: 0'], axis=1, inplace=True)

In [5]:
train.head()

,CLIENT,DATA_TRANS,AZS_NUMBER,REGION_AZS,VID_NP,COL_LITR,CENA_CLIENT,day,month,year,daytime_in_seconds,rounded_client_price,REGION_AZS_mean_target,VID_NP_mean_target,rounded_client_price_mean_target,month_mean_target,year_mean_target,AZS_NUMBER_mean_target,CLIENT_mean_target
0,46219,01.11.2016 0:19:47,2335,3,1,15.00,35.70,1,11,2016,1187,35.7,51.288309,138.503952,77.493838,82.864377,78.124254,48.362749,53.476351
1,33792,01.11.2016 1:32:50,3356,18,2,30.00,31.43,1,11,2016,5570,31.4,62.550582,32.530679,82.804494,82.864377,78.124254,36.637372,60.074300
2,16215,01.11.2016 3:35:38,3381,18,1,220.00,32.93,1,11,2016,12938,32.9,62.550582,138.503952,114.982810,82.864377,78.124254,170.101413,73.436525
3,33792,01.11.2016 1:12:55,3303,18,2,15.00,31.16,1,11,2016,4375,31.1,62.550582,32.530679,104.272030,82.864377,78.124254,57.409964,60.074300
4,17075,01.11.2016 2:38:13,3276,18,1,133.67,34.72,1,11,2016,9493,34.7,62.550582,138.503952,73.190038,82.864377,78.124254,55.415038,173.086905


In [ ]:
train["holidays"] = 0
train.loc[[train["day"] =<  8][train["month"]==1 ],"holidays"] = 1
train.loc[[train["day"] == 23][train["month"]==2 ],"holidays"] = 1
train.loc[[train["day"] ==  8][train["month"]==3 ],"holidays"] = 1
train.loc[[train["day"] ==  1][train["month"]==5 ],"holidays"] = 1
train.loc[[train["day"] ==  9][train["month"]==5 ],"holidays"] = 1
train.loc[[train["day"] == 12][train["month"]==6 ],"holidays"] = 1
train.loc[[train["day"] ==  4][train["month"]==11],"holidays"] = 1

test["holidays"] = 0
test.loc[[test["day"] =<  8][test["month"]==1 ],"holidays"] = 1
test.loc[[test["day"] == 23][test["month"]==2 ],"holidays"] = 1
test.loc[[test["day"] ==  8][test["month"]==3 ],"holidays"] = 1
test.loc[[test["day"] ==  1][test["month"]==5 ],"holidays"] = 1
test.loc[[test["day"] ==  9][test["month"]==5 ],"holidays"] = 1
test.loc[[test["day"] == 12][test["month"]==6 ],"holidays"] = 1
test.loc[[test["day"] ==  4][test["month"]==11],"holidays"] = 1

In [6]:
def data_and_time(time):#Функция извлекающая дату и время(вроде работает)
    if len(time)>=18:
        return int(time[-8:-6])*3600 + int(time[-5:-3])*60 + int(time[-2:])
    else:
        return 0

In [7]:
%%time
train["day"] = train.DATA_TRANS.apply(lambda x:int(x[:2]))
print("ok")
train["month"] = train.DATA_TRANS.apply(lambda x:int(x[3:5]))
train["year"] = train.DATA_TRANS.apply(lambda x:int(x[6:10]))
train["daytime_in_seconds"] = train.DATA_TRANS.apply(data_and_time)
print("ok")

test["day"] = test.DATA_TRANS.apply(lambda x:int(x[:2]))
test["month"] = test.DATA_TRANS.apply(lambda x:int(x[3:5]))
test["year"] = test.DATA_TRANS.apply(lambda x:int(x[6:10]))
print("ok")
test["daytime_in_seconds"] = test.DATA_TRANS.apply(data_and_time)

ok
ok
ok
CPU times: user 3min 18s, sys: 6.34 s, total: 3min 24s
Wall time: 2min 57s


In [8]:
train.to_csv("./train_with_data_and_mean.csv")#Считается долго, поэтому сохраняем новые данные 
test.to_csv("./test_with_data_and_mean.csv")

In [5]:
def likelyhood_mean_target_encoding(df, test, columns, alpha=10):
    """
    Это имплементация алгоритма Стаса Семенова, должна очень сильно увеличивать скор
    Мы берем среднее значение по каждому объекту в категории, 
    но плюс к этому сглаживаем значения для очень редких категорий, за степень сглаживания отвечает alpha
    columns - список колонок с категориями
    df - train
    test - test
    """
    for col in columns:  
        count = 0
        global_mean = df.COL_LITR.mean()
        df[col+"_mean_target"] = global_mean
        test[col+"_mean_target"] = global_mean
        print(col)
        print(len(set(df[col])))
        
        
        for i in set(df[col]):
            count += 1
            cn = len(df[col][df[col] == i])
            value = (df["COL_LITR"][df[col] == i].mean()*cn + global_mean*alpha) / (cn + alpha)
            df[col+"_mean_target"][df[col] == i] = value
            test[col+"_mean_target"][test[col] == i] = value
            if count%100==0:
                print(count)
                
        print("ok")
        df.to_csv("./train_with_mean.csv") #Считается долго, поэтому сохраняем новые данные 
        test.to_csv("./test_with_mean.csv")
                
    return "ok"

In [6]:
def likelyhood_mean_target_encoding_with_loc(df, test, columns, alpha=10):
    """
    Это имплементация алгоритма Стаса Семенова, должна очень сильно увеличивать скор
    Мы берем среднее значение по каждому объекту в категории, 
    но плюс к этому сглаживаем значения для очень редких категорий, за степень сглаживания отвечает alpha
    columns - список колонок с категориями
    df - train
    test - test
    """
    for col in columns:  
        count = 0
        global_mean = df.COL_LITR.mean()
        df.loc[:,[col+"_mean_target"]] = global_mean
        test.loc[:,[col+"_mean_target"]] = global_mean
        print(col)
        print(len(set(df[col])))
        
        
        for i in set(df[col]):
            count += 1
            cn = len(df.loc[df.loc[:,col] == i,col])
            value = (df.loc[df.loc[:,col] == i,["COL_LITR"]].mean()*cn + global_mean*alpha) / (cn + alpha)
            
            df.loc[df.loc[:,col] == i,col+"_mean_target"] = value
            test.loc[test.loc[:,col] == i,col+"_mean_target"] = value
            if count%100==0:
                print(count)
                
        print("ok")
        df.to_csv("./train_with_mean.csv") #Считается долго, поэтому сохраняем новые данные 
        test.to_csv("./test_with_mean.csv")
                
    return "ok"

In [7]:
train["rounded_client_price"] = (train.CENA_CLIENT * 10 // 1 / 10)
test["rounded_client_price"] = (test.CENA_CLIENT * 10 // 1 / 10)

In [8]:
train.head()

,CLIENT,DATA_TRANS,AZS_NUMBER,REGION_AZS,VID_NP,COL_LITR,CENA_CLIENT,day,month,year,daytime_in_seconds,rounded_client_price
0,46219,01.11.2016 0:19:47,2335,3,1,15.00,35.70,1,11,2016,1187,35.7
1,33792,01.11.2016 1:32:50,3356,18,2,30.00,31.43,1,11,2016,5570,31.4
2,16215,01.11.2016 3:35:38,3381,18,1,220.00,32.93,1,11,2016,12938,32.9
3,33792,01.11.2016 1:12:55,3303,18,2,15.00,31.16,1,11,2016,4375,31.1
4,17075,01.11.2016 2:38:13,3276,18,1,133.67,34.72,1,11,2016,9493,34.7


In [9]:
%%time
likelyhood_mean_target_encoding(train, test, ["REGION_AZS","VID_NP", "rounded_client_price", "month", "year", "AZS_NUMBER"])

REGION_AZS


/opt/intel/intelpython3/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/intel/intelpython3/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


26
ok
VID_NP
3
ok
rounded_client_price
499
100
200
300
400
ok
month
12
ok
year
2
ok
AZS_NUMBER
4094
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
ok


KeyboardInterrupt: 

In [17]:
%%time
likelyhood_mean_target_encoding(train, test, ["CLIENT"], alpha=5)

CLIENT
42686


/opt/intel/intelpython3/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/intel/intelpython3/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
1850

'ok'

In [18]:
print("Finished")

Finished


In [19]:
train.head()

,CLIENT,DATA_TRANS,AZS_NUMBER,REGION_AZS,VID_NP,COL_LITR,CENA_CLIENT,day,month,year,daytime_in_seconds,rounded_client_price,REGION_AZS_mean_target,VID_NP_mean_target,rounded_client_price_mean_target,month_mean_target,year_mean_target,AZS_NUMBER_mean_target,CLIENT_mean_target
0,46219,01.11.2016 0:19:47,2335,3,1,15.00,35.70,1,11,2016,1187,35.7,51.288309,138.503952,77.493838,82.864377,78.124254,48.362749,53.476351
1,33792,01.11.2016 1:32:50,3356,18,2,30.00,31.43,1,11,2016,5570,31.4,62.550582,32.530679,82.804494,82.864377,78.124254,36.637372,60.074300
2,16215,01.11.2016 3:35:38,3381,18,1,220.00,32.93,1,11,2016,12938,32.9,62.550582,138.503952,114.982810,82.864377,78.124254,170.101413,73.436525
3,33792,01.11.2016 1:12:55,3303,18,2,15.00,31.16,1,11,2016,4375,31.1,62.550582,32.530679,104.272030,82.864377,78.124254,57.409964,60.074300
4,17075,01.11.2016 2:38:13,3276,18,1,133.67,34.72,1,11,2016,9493,34.7,62.550582,138.503952,73.190038,82.864377,78.124254,55.415038,173.086905


In [20]:
print("теперь точно Finished")

теперь точно Finished
